# Kaggle Challenge: Detección de elementos patrimoniales





In [1]:
import tensorflow as tf
from keras.optimizers import SGD,RMSprop,Adamax,Adam
from keras.layers.normalization import BatchNormalization
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LeakyReLU,GaussianDropout,GlobalMaxPooling2D,SpatialDropout2D
from pathlib import Path
from PIL import Image
from keras.utils import to_categorical, plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.initializers import he_uniform
from keras.constraints import max_norm
from contextlib import redirect_stdout
from keras_preprocessing.image import ImageDataGenerator

KeyboardInterrupt: 






































# Funciones a utilizar

*image_set_generator* Es una función que toma cada imagen, la transforma en un arreglo tri-dimensional y luego la almacena en una lista que posteriormente es transformada en arreglo para su posterior entrenamiento o prediccion según seal el caso.

Parámetros:

X: Corresponde al arreglo que contenga los nombres de las imagenes a utilizar.
image_path: Corresponde a un string que contiene la direccion de donde se encuentran (carpeta)
image_format: La extensión del archivo, se añadio para poder recibir más tipos de imagenes.

In [ ]:
def image_set_generator(X,image_path,image_format):
    gen = []
    for i in X:
        image =  tf.keras.preprocessing.image.load_img(image_path+i+'.'+image_format,target_size=None)
        data =keras.preprocessing.image.img_to_array(image)
        gen.append(data)
       
    return np.asarray(gen)

cat_encode/decode: Este par de funciones transforma el arreglo de etiquetas en categorias binarias para mejorar así su representación y evitar problemas de entrenamiento del modelo. 

Parámetros:

Data: informacion a transformar o revertir transformacion (lista o arreglo)
cat: conjunto de categorias (lista o arreglo)

In [ ]:
def cat_encode(data,cat):
    data=list(data)
    cat=list(cat)
    for i in range(len(data)):
        for j in range(len(cat)):
            if data[i]==cat[j]:
                data[i]=j
                break
            else:
                continue
    return keras.utils.to_categorical(data,len(cat))
    #return np.asarray(data)
def cat_decode(data,cat):
    #Y=data
    Y= [np.argmax(y, axis=None, out=None) for y in data]
    cat=list(cat)
    
    for i in range(len(Y)):
        Y[i]=cat[Y[i]]
    return Y

Autopredict automatiza el proceso de prediccion y guardado en un archivo csv. Además incorpora una opcion para mostrar resultados

auto_predict(data_x,data_y,cat,example_show):

data_x: corresponde al conjunto de datos que se busca predecir (X_train)
data_set: corresponde al conjunto de datos que se ingresará en el archivo csv
cat:categorias
example_show: True/False si se quiere adjuntar un muestreo de los primeros 20 resultados de la prediccion.

In [ ]:
def auto_predict(data_x,data_set,cat,example_show):
    pred = model.predict(data_x)
    y_test = cat_decode(pred,cat)
    data_set['Expected']= y_test
    data_set.to_csv('sample_submission_1234.csv',columns=['Id','Expected'],index=False)
    if example_show==True:
        return print("Exito!\n",y_test[0:20])
    else:
        return print("Exito")

# Cargando la data

In [ ]:
train_labels = pd.read_csv('./data/train_labels.csv')
test_labels = pd.read_csv('./data/sample_submission.csv')
train_path = './data/train_images/'
test_path = './data/test_images/'
X_names = train_labels["Id"].values
X_train = image_set_generator(X_names,train_path,'jpg')


Y_train = train_labels["Expected"]
cat = Y_train.unique()
Y_train=cat_encode(Y_train,cat)
Y_train = np.asarray(Y_train)
X_test_names = test_labels["Id"].values
X_test  = image_set_generator(X_test_names,test_path,'jpg')

#X_train =  preprocessing.scale(X_train)
#X_test = preprocessing.scale(X_test)

# Data augmentation

In [ ]:
#comentar solo en caso de que se prueben modelos 1 y 2
datagen = ImageDataGenerator(
    featurewise_center=True,
    samplewise_center=True,
    samplewise_std_normalization=True,
    featurewise_std_normalization=True,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=25,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    validation_split=0.3)
datagen.fit(X_train)
train_generator = datagen.flow(
    X_train,Y_train,
    batch_size=64,
    shuffle=True,
    sample_weight=None,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset="training",
)
val_generator= datagen.flow(
    X_train,Y_train,
    batch_size=64,
    shuffle=True,
    sample_weight=None,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset="validation",
)


In [ ]:
####                              ENCODER TEST                         ####

#encoder/decoder, quitar comentarios solo en caso de prueba. SPOILER: FUNCIONAN!
#print(Y_train)
#Y_train=cat_encode(Y_train,cat)
#Y_train=cat_decode(Y_train,cat)
#print(Y_train)

# Implementando el modelo (1/x RELUciente) (1,2 y 3)

65% Sin data augmentation, usando SGD ( con la configuracion mostrada abajo)

Test 0.912775993347168

In [ ]:
model = Sequential()
model.add(BatchNormalization(input_shape=(128,128,3) ))
#model.add(SpatialDropout2D(0.1))

model.add(Conv2D(4,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.03))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(SpatialDropout2D(0.1))

model.add(Conv2D(16,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.03))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(SpatialDropout2D(0.2))



#model.add(SpatialDropout2D(0.3))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.02))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(SpatialDropout2D(0.4))

#model.add(Conv2D(256,(5,5), padding='same'))
#model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.03))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(SpatialDropout2D(0.45))

model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.03))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(SpatialDropout2D(0.5))
model.add(Flatten())
#model.add(GaussianDropout(rate = 0.2))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.03))

model.add(Dropout(0.3))




model.add(Dense(10,activation='softmax'))

In [ ]:
model.summary()

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

## Callbacks y otras configuraciones para compiler

In [ ]:
####################################        NUEVO                    #######################################
#Nota: Para el tercer modelo (test) con data augmentation
    
    
#########################################           CALLBACKS           #########################################
early_stop = EarlyStopping(monitor='val_categorical_accuracy', patience=12)

check_point = ModelCheckpoint(
    filepath='/tmp/checkpoint',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,    
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='loss', factor=0.85, patience=7, verbose=0, mode='auto',
    min_delta=0.0001, cooldown=0, min_lr=0.00006
)

#########################################           OPTIMIZER           #########################################
opt=Adam(amsgrad=True)
########################################           LOSS FUCTION           #######################################

#########################################           Metrics           #########################################


## Compilado y entrenamiento de nuestro modelo

In [ ]:
train = model.fit(train_generator,epochs=120,validation_data= val_generator,callbacks=[early_stop,check_point,reduce_lr])
model.load_weights('/tmp/checkpoint')

In [ ]:
history=train.history
plt.plot(history['categorical_accuracy'],'r',label = 'categorical_accuracy')

# Plot the validation loss
plt.plot(history['val_categorical_accuracy'],label = 'val_categorical_accuracy')

# Show the figure
plt.show()

In [ ]:
train_data = pd.DataFrame(train.history)
train_data['epoch'] = train.epoch
train_data.to_excel('train_data_model3.xlsx',index=False)
train_data

In [ ]:
val_categorical_accuracy = max(list(train_data['val_categorical_accuracy']))
val_categorical_accuracy

In [ ]:
auto_predict(X_test,test_labels,cat,True)

# Modelo 4:  MatNetV6 (Mejor en KAGGLE)

In [ ]:
n = 35
r = 2.2
N = n

model = Sequential() 
model.add(Conv2D(n, (2, 2),input_shape=(128,128,3), activation='relu', padding='same')) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) 

for i in range(5):
    if i < 4: n = round(r*n)       
    model.add(Conv2D(n, (2, 2), activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) 
model.add(Dense(80,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10, activation = 'softmax'))

model.summary()

# Compilado

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='Adam', 
              metrics=['accuracy','categorical_accuracy']) 

In [ ]:
# Callbacks
early_stop = EarlyStopping(monitor='val_categorical_accuracy', patience=12)

check_point = ModelCheckpoint(
    filepath='/tmp/checkpoint',
    save_weights_only=False,
    monitor='val_categorical_accuracy',
    mode='max',
    save_best_only=True,    
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_categorical_accuracy', factor=0.4, patience=7, verbose=0, mode='auto',
    min_delta=0.0001, cooldown=0, min_lr=0.0001
)

# Entrenamiento
train = model.fit(X_train,Y_train,epochs=120,validation_split=0.3,batch_size=64,shuffle=True,callbacks=[early_stop,check_point,reduce_lr])
model.load_weights('/tmp/checkpoint')

In [ ]:
train_data = pd.DataFrame(train.history)
train_data['epoch'] = train.epoch
train_data.to_excel('train_data.xlsx',index=False)
train_data

In [ ]:
val_categorical_accuracy = max(list(train_data['val_categorical_accuracy']))
val_categorical_accuracy

In [ ]:
def plot_model_history(train_data): 
    sns.set_style('darkgrid')
    fig, axs = plt.subplots(1,2,figsize=(15,5)) 
    # summarize history for accuracy
    axs[0].plot(train_data['accuracy']) 
    axs[0].plot(train_data['val_accuracy']) 
    axs[0].set_title('Model Categorical Accuracy')
    axs[0].set_ylabel('Accuracy') 
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['train', 'validate'], loc='upper left')
    # summarize history for loss
    axs[1].plot(train_data['loss']) 
    axs[1].plot(train_data['val_loss']) 
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss') 
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['train', 'validate'], loc='upper left')
    plt.show()

plot_model_history(train_data)

# Predicción

In [ ]:
pred = model.predict(X_test)
y_pred = cat_decode(pred,cat)

test_labels['Expected']= y_pred
test_labels.to_csv('sample_submission_mateneuronas.csv',columns=['Id','Expected'],index=False)
test_labels.describe()

# Recurrencias en predicciones

In [ ]:
test_labels.groupby('Expected').count()

# Guardar modelo

In [ ]:

# Modelo
model.save('model.h5')
# Summary
with open('model_summary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

# Información del modelo

In [ ]:
file = open("model_info.txt","w") 
texto = f"""\
MODELO
- 6 capas convolucionales
- Arquitectura tipo cono, n = {N}, r = {r}
- 1 capas densa de 80 neuronas
- Activaciones Relu
- Batch Normalization
- Dropout = 0.2
- Parámetros = {model.count_params()}

ENTRENAMIENTO
- Optimizador Adam
- Checkpoint: monitor = val_categorical_accuracy
- Early stopping: monitor = val_categorical_accuracy, patience = 12
- Reduce LR: factor = 0.4, patience = 7

MÉTRICAS
- val_categorical_accuracy = {val_categorical_accuracy}
"""
file.write(texto)
file.close() 